# Compositional Analysis - Timepoint Comparison

### Environment Set Up

In [ ]:
# load libraries
import warnings
import matplotlib.pyplot as plt
import mudata as mu
import pertpy as pt
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import os
from statannot import add_stat_annotation

In [ ]:
# set up figure parameters
plt.rcParams['figure.figsize'] = (6, 4)
sc.settings.verbosity = 0
sc.settings.set_figure_params(
    figsize=(4,4),
    dpi=300,
    facecolor="white",
    frameon=False,
)

In [ ]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set up dirs
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"
fig_dir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "timepoint")
sc.settings.figdir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "timepoint")

In [ ]:
# read anndata object
adata = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_TNK_annotated_13-04-24.h5ad"))

In [ ]:
# create condition variable
adata

## Model Setup & Inference

In [ ]:
# initiate scCODA model
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="Annotation_2.0",
    sample_identifier="sample",
    covariate_obs=["subproject", "patient", "timepoint", "response", "ICI_status", "Condition"],
        
)
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
pt.pl.coda.boxplots(sccoda_data, 
                    modality_key="coda", 
                    feature_name="timepoint", 
                    add_dots=True,
                    #figsize=[6,6],
                    level_order=["SCR", "C02"],
                    cmap=["coral","darkviolet"]
                   )
plt.show()

In [ ]:
# prepare the model
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula="Condition", #condition = timepoint
    reference_cell_type="automatic",
)
sccoda_data["coda"]

In [ ]:
# run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda")
sccoda_data["coda"]

## Result Intepretation

In [ ]:
# see most relevant information
sccoda_model.summary(sccoda_data, modality_key="coda")

In [ ]:
sccoda_model.credible_effects(sccoda_data, modality_key="coda")

### Adjust FDR

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="coda", est_fdr=0.2)
sccoda_model.summary(sccoda_data, modality_key="coda")

## Save Results

In [ ]:
#sccoda_data.write_h5mu(os.path.join(work_dir, "data", "outputdata", "combined", "TNK_Combined_SCR_CO2_immune_scCODA_15-04-24.h5mu"))

In [ ]:
test_model = pt.tl.Sccoda()
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
effect = test_model.get_effect_df(sccoda_data, modality_key="coda")
effect

In [ ]:
sccoda_data["coda"].varm["intercept_df"]

## Visualization

In [ ]:
# proportion
props = round(sccoda_data['coda'].var.n_cells / sccoda_data['coda'].var.n_cells.sum(), 3)
props

In [ ]:
tab20c = plt.get_cmap('tab20c')
palette = [tab20c(i) for i in range(tab20c.N)]

plt.figure(figsize=(6,.5), dpi=120)

cf=props.cumsum()

#plt.barh(0, props[0], edgecolor="white")

#for i in range(1, len(props)):
#    plt.barh(0, props[i], bottom=cf[i-1], edgecolor="white", color=palette[i])

plt.barh(0, props[0], edgecolor="white", height=0.01)

for i in range(1, len(props)):
    plt.barh(0, props[i], left=cf[i-1], edgecolor="white", height=0.01, color=palette[i])

# Remove y-axis ticks and labels
plt.gca().axes.get_yaxis().set_visible(False)

# Set x-axis label and title
plt.xlabel('Proportion')
# Move the x-axis label and title below the plot
plt.gca().xaxis.set_label_coords(0.5, -1)
plt.gca().title.set_position([0.5, -0.2])
plt.xticks(fontsize=11)

#plt.ylabel('proportion')
#frame1=plt.gca()
#frame1.axes.get_xaxis().set_visible(False)
plt.xlim(0, 1.0)

plt.savefig("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/combined/TNK/compositional_analysis/barplot_all_celltypes.pdf", dpi=600, format="pdf", bbox_inches="tight")
plt.savefig("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/ISCO24_poster/Fig2/barplot_all_celltypes.pdf", dpi=600, format="pdf", bbox_inches="tight")
plt.show()

In [ ]:
import matplotlib.colors as mcolors

tab20c = plt.get_cmap('tab20c')
palette = [tab20c(i) for i in range(tab20c.N)]
print(palette)
hex_colors = [mcolors.to_hex((r, g, b)) for r, g, b, a in palette]
print(hex_colors)

In [ ]:
plt.figure(figsize=(.5, 6), dpi=120)

cf = np.cumsum(props)

# Plot the first bar segment
plt.bar(0, props[0], edgecolor="white", width=0.01)

# Plot the subsequent bar segments on top of each other
for i in range(1, len(props)):
    plt.bar(0, props[i], bottom=cf[i-1], edgecolor="white", width=0.01)

# Remove y-axis ticks and labels
plt.gca().axes.get_yaxis().set_visible(True)
plt.gca().axes.get_xaxis().set_visible(False)
# Set x-axis label and title
plt.xlabel('Proportion')
plt.yticks(fontsize=10)

# Save the figure
plt.savefig("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/combined/TNK/compositional_analysis/barplot_all_celltypes.png", dpi=600, format="png", bbox_inches="tight")
plt.savefig("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/ISCO24_poster/Fig2/barplot_all_celltypes.png", dpi=600, format="png", bbox_inches="tight")

plt.show()

In [ ]:
# visualize effects (significant changes in cell type composition)
plot=pt.pl.coda.effects_barplot(sccoda_data, 
                           modality_key="coda", 
                           #parameter="timepoint",
                           #figsize=[3,3],
                           dpi=300)
plt.xticks(fontsize=8) 
plt.yticks(fontsize=10)  
plt.savefig(os.path.join(work_dir, "figures", "TFM", "Fig2", "TNK_Comp_Significant_Log2FC_timepoint_all.pdf"), dpi=300, format="pdf", bbox_inches="tight")

In [ ]:
# define cell types that have an effect
effect_cell_types = ["CD4 T Central Memory", "CD4 T Follicular Helper", "CD8 T TRM PreExhausted"]

In [ ]:
sccoda_data['coda'].var

In [ ]:
# some booxplot arguments to customize the plot
boxprops = dict(linestyle='-', linewidth=1, edgecolor='black')
flierprops = dict(marker='o', markerfacecolor='none', #markersize=12,
                  markeredgecolor='black')
medianprops = dict(linestyle='-', linewidth=.8, color='black')
#meanpointprops = dict(marker='D', markeredgecolor='black',markerfacecolor='black')
meanlineprops = dict(linestyle='--', linewidth=.8, color='black')


In [ ]:
# define cell types for plotting
cell_types = sccoda_data['coda'].var_names.tolist()
print(cell_types)

In [ ]:
# create df for plotting
feature_name=["Condition", "response", "patient"]
data=sccoda_data['coda']

sample_sums = np.sum(data.X, axis=1, keepdims=True)
X = data.X/sample_sums
value_name = "Proportion"

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
        merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)
if cell_types is not None:
    plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

In [ ]:
plot_df

In [ ]:
# some booxplot arguments to customize the plot
import seaborn
boxprops = dict(linestyle='-', linewidth=1, edgecolor='black')
flierprops = dict(marker='o', markerfacecolor='black', #markersize=12,
                  markeredgecolor='none')
medianprops = dict(linestyle='-', linewidth=.8, color='black')
#meanpointprops = dict(marker='D', markeredgecolor='black',markerfacecolor='black')
meanlineprops = dict(linestyle='--', linewidth=.8, color='black')


import seaborn as sns
plt.rcParams.update({'font.size': 13})
seaborn.set_style(style='white') 

#custom_order = ['CD4 T', 'CD8 T', 'NK', 'Myeloid', 'pDC', 'Plasma', 'B Cell']


fig, ax = plt.subplots(figsize=(7,5), dpi=120)

ax = sns.boxplot(
    x="Cell type", 
    y="Proportion", 
    data=plot_df,
    hue="Condition", 
    #liersize=1,
    palette=["coral","darkviolet"], 
    #order=custom_order,
    ax=ax,
    vert=True, 
    legend=False,
    patch_artist=True, 
    meanline=True, 
    showmeans=True,
    showfliers=False,
    boxprops=boxprops,
    #flierprops=flierprops,
    medianprops=medianprops,
    meanprops=meanlineprops
)

'''
sns.stripplot(
    x="Cell type", 
    y="Proportion", 
    data=plot_df, 
    #style="Patient",
    hue="Condition",  # Color by "patient" variable
    palette=["coral","darkviolet"],  # Choose a color palette
    dodge=True,  # Separate dots for each level of "patient
    #order=custom_order,
    #jitter=False
    ax=ax,
    size=5,
    edgecolor="black",
    linewidth=.5,
    alpha=1,
)
'''
plt.ylim(0, 0.3)

#ax.set_xticks([])
ax.set_xlabel('Cell type')
plt.yticks(fontsize = 10, ) 
plt.xticks(fontsize = 11, rotation = 90) 

handles, labels = ax.get_legend_handles_labels()
#ax.legend(handles[:2], labels[:2], title='Timepoint', bbox_to_anchor=(1, 1))

#legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Patient", frameon=True)
plt.tight_layout()
        
plt.savefig(os.path.join(work_dir, "figures", "TFM", "Fig2", "Boxplots_cell_type_tiempoints_comparison.pdf"), dpi=600, format="pdf", bbox_inches="tight")


In [ ]:
# plot for Poster/Thesis figures
# overview of cell type distribution across patients
for cell_type in cell_types:

    if cell_type in effect_cell_types:
        
        
        fig, ax = plt.subplots(figsize=(2,2), dpi=120)
    
        sns.boxplot(
            x="patient", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type],
            hue="patient", 
            #liersize=1,
            palette="Spectral", 
            #order=["T0/-ICI","T1/+ICI"],
            ax=ax,
            vert=True, 
            patch_artist=True, 
            meanline=True, 
            showmeans=True,
            showfliers=False,
            boxprops=boxprops,
            #flierprops=flierprops,
            medianprops=medianprops,
            meanprops=meanlineprops
        )
            
        
        sns.swarmplot(
            x="patient", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type], 
            hue="Condition",  # Color by "patient" variable
            palette=["coral", "darkviolet"],  # Choose a color palette
            dodge=False,  # Separate dots for each level of "patient
            #jitter=False
            ax=ax,
            size=4,
            edgecolor="black",
            linewidth=.5,
            alpha=1
        )
    
        #ax.set_xticks([])
        plt.title(cell_type, fontweight="bold", pad=20)
        plt.xlabel('Patient', labelpad=10)
        plt.ylabel('Proportion', labelpad=10)
        plt.yticks(fontsize = 10) 
        plt.xticks(fontsize = 10) 
        #plt.set_xticks([0,1,2,3,4], ["PD_01", "PD_02", "PD_03", "SD_01", "PD_04"], ha="center", rotation=30)

        
        legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Condition", frameon=True, 
                           labels=["PD_01", "PD_02", "PD_03", "SD_01", "PD_04"])
        plt.setp(legend.get_title(),fontsize='12')
        plt.tight_layout()

        
        plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/ISCO24_poster/Fig2", "TNK_Comp_"+cell_type+"_boxplot_by_patient.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
# overview of cell type distribution across patients
for cell_type in cell_types:

    if cell_type in effect_cell_types:
        significance = "*"
    else:
        significance = "ns"
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)

    sns.boxplot(
        x="patient", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="patient", 
        #liersize=1,
        palette="Spectral", 
        #order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="patient", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="Condition",  # Color by "patient" variable
        palette=["coral", "darkviolet"],  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=4,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type+" - "+significance)
    ax.set_xlabel('Patient')
    plt.yticks(fontsize = 10) 
    plt.xticks(fontsize = 10) 
    
    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Condition", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
      
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated" 
    
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_patient.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
plot = pt.pl.coda.boxplots(
        sccoda_data,
        modality_key="coda",
        feature_name="patient",
        figsize=(8,4),
        add_dots=False,
        plot_facets=False, 
        args_boxplot={"vert":True, 
                    "patch_artist":True, 
                    "meanline":True, 
                    "showmeans":True,
                    "boxprops":boxprops,
                    "flierprops":flierprops,
                    "medianprops":medianprops,
                    "meanprops":meanlineprops,
                    },
        cmap="Spectral"
        )
plt.xticks(fontsize=10)
plt.savefig(os.path.join(fig_dir, "Comp_all_boxplot_by_patient.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
effect["Final Parameter"]

In [ ]:
# overview of cell type distribution across timepoints

plt.rcParams.update({'font.size': 13})

for cell_type in cell_types:

    if cell_type in effect_cell_types:
        significance = "*"
    else:
        significance = "ns"
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)
    
    sns.boxplot(
        x="Condition", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="Condition", 
        #liersize=1,
        palette=["coral","darkviolet"], 
        order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="Condition", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="patient",  # Color by "patient" variable
        palette="Spectral",  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=5,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type+" - "+significance)
    ax.set_xlabel('ICI Response')
    plt.yticks(fontsize = 10) 
    plt.xticks(fontsize = 10) 
    
    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Patient", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
    
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated"

        
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_condition.png"), dpi=600, format="png", bbox_inches="tight")


In [ ]:
# create new_patient column in plot_df
patient_mapping = {
    '01': 'PD_01',
    '02': 'PD_02',
    '03': 'PD_03',
    '08': 'SD_01',
    '10': 'PD_04'
}
plot_df['new_patient'] = plot_df['patient'].map(patient_mapping)
print(plot_df)

In [ ]:
# plots for Poster/Thesis Figures
# overview of cell type distribution across timepoints
boxprops = dict(linestyle='-', linewidth=1, edgecolor='black', alpha=0.7)
plt.rcParams.update({'font.size': 13})
sns.set_style(rc = {'axes.facecolor': 'white'})

#effect_cell_types.append("CD56hi CD16lo NK")

for cell_type in cell_types:

    #if cell_type in effect_cell_types:
    if 1 == True:
    
        fig, ax = plt.subplots(figsize=(2,2), dpi=120)
        
        sns.boxplot(
            x="Condition", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type],
            hue="Condition", 
            #liersize=1,
            palette=["coral","darkviolet"], 
            order=["T0/-ICI","T1/+ICI"],
            ax=ax,
            vert=True, 
            patch_artist=True, 
            meanline=True, 
            showmeans=True,
            showfliers=False,
            boxprops=boxprops,
            #flierprops=flierprops,
            medianprops=medianprops,
            meanprops=meanlineprops
        )
            
        
        sns.swarmplot(
            x="Condition", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type], 
            hue="patient",  # Color by "patient" variable
            palette="Spectral",  # Choose a color palette
            dodge=False,  # Separate dots for each level of "patient
            #jitter=False
            ax=ax,
            size=5,
            edgecolor="black",
            linewidth=.5,
            alpha=1
        )
        '''
        sns.pointplot(
            data=plot_df[plot_df["Cell type"] == cell_type], 
            x="Condition", 
            y="Proportion", 
            hue="patient", 
            legend= False,
            palette="Spectral",
            linewidth=1.5,
            errwidth=1,
            markers='|',
            #errorbar=None,
            join=True
        )
        '''
        plt.title(cell_type, pad=18)
        plt.xlabel('', labelpad=10)
        plt.ylabel('', labelpad=10)
        plt.yticks(fontsize = 10) 
        plt.xticks(fontsize = 10) 
        #plt.set_xticks([0,1,2,3,4], ["PD_01", "PD_02", "PD_03", "SD_01", "PD_04"], ha="center", rotation=30)
    
            
        legend = plt.legend(loc='right', bbox_to_anchor=(2, .5), ncol=1, title="Response", frameon=True, facecolor="white")
        plt.setp(legend.get_title(),fontsize='12')
        plt.tight_layout()
  
        xmin, xmax, ymin, ymax = plt.axis()
        plt.ylim((0, ymax + (0.66)*ymax))

        if cell_type == "CD4 T CM/EarlyActivated":
            cell_type = "CD4 T CM-EarlyActivated"
        
        plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/TFM/Fig2", "TNK_Comp_"+cell_type+"_boxplot_by_condition_colby_response.pdf"), dpi=600, format="pdf", bbox_inches="tight")

In [ ]:
print(sccoda_data['coda'].X.shape)
print(sccoda_data['coda'].obs.shape)

In [ ]:
patients = ["01", "02", "03", "08", "10"]
patient_colors = seaborn.color_palette("Spectral", n_colors=len(patients))
patient_color_dict = dict(zip(patients, patient_colors))

In [ ]:
# stacked plot of cell type proportions per patient
sns.set_style(rc = {'axes.facecolor': 'white'})
plot = pt.pl.coda.stacked_barplot(
    sccoda_data, 
    modality_key="coda", 
    feature_name="patient",
    dpi=300
)
plot.set(title = "Patient")
plot.set_xticks([0,1,2,3,4], ["P01", "P02", "P03", "P04", "P05"], ha="center", rotation=0)
legend = plt.legend(loc='right', bbox_to_anchor=(1.7, .5), ncol=1, title="Cell Type", frameon=True)
plt.setp(legend.get_texts(),fontsize='8')
plt.setp(legend.get_title(),fontsize='12')
plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/TFM/Fig2", "TNK_Comp_immune_barplot_by_patient.png"), dpi=600, format="png", bbox_inches="tight")
plt.show()

In [ ]:
# stacked plot of cell type proportions per patient
plot = pt.pl.coda.stacked_barplot(
    sccoda_data, 
    modality_key="coda", 
    feature_name="patient",
    dpi=300
)
plot.set(title = "Patient")
plot.set_xticks([0,1,2,3,4], ["P01", "P02", "P03", "P04", "P05"], ha="center", rotation=0)
legend = plt.legend(loc='right', bbox_to_anchor=(1.7, .5), ncol=1, title="Cell Type", frameon=True)
plt.setp(legend.get_texts(),fontsize='8')
plt.setp(legend.get_title(),fontsize='12')
plt.show()
# P08 = P04
# P10 = P05

In [ ]:
# stacked plot of cell type proportions per timepoint
plot = pt.pl.coda.stacked_barplot(
    sccoda_data, modality_key="coda", 
    feature_name="Condition", 
    level_order=["T0/-ICI", "T1/+ICI"],
    dpi=300
)
plot.set(title = "Timepoint")
plot.set_xticks([0,1], ["T0/-ICI", "T1/+ICI"], rotation = 0, rotation_mode="anchor", ha="center")
legend = plt.legend(loc='right', bbox_to_anchor=(1.7, .5), ncol=1, title="Cell Type", frameon=True)
plt.setp(legend.get_texts(),fontsize='8')
plt.setp(legend.get_title(),fontsize='12')
plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/TFM/Fig2", "TNK_Comp_immune_barplot_by_condition.png"), dpi=600, format="png", bbox_inches="tight")
plt.show()

In [ ]:
# stacked plot of cell type proportions per timepoint
plot = pt.pl.coda.stacked_barplot(
    sccoda_data, modality_key="coda", 
    feature_name="timepoint", 
    level_order=["SCR", "C02"], 
    dpi=300
)
plot.set(title = "Timepoint")
plot.set_xticks([0,1], ["T0", "T1"], rotation = 0, rotation_mode="anchor", ha="center")
legend = plt.legend(loc='right', bbox_to_anchor=(1.7, .5), ncol=1, title="Cell Type", frameon=True)
plt.setp(legend.get_texts(),fontsize='8')
plt.setp(legend.get_title(),fontsize='12')
plt.savefig(os.path.join(fig_dir, "TNK_Comp_immune_barplot_by_timepoint.png"), dpi=600, format="png", bbox_inches="tight")

plt.show()

In [ ]:
# stacked plot of cell type proportions per ICI status
plot = pt.pl.coda.stacked_barplot(
    sccoda_data, 
    modality_key="coda", 
    feature_name="ICI_status",
    #level_order=["`-ICI`", "`+ICI/PD`", "`+ICI/PD`"], 
    dpi=300
)
plot.set(title = "ICI Status")
plot.set_xticks([0,1,2], ["+ICI/PD", "+ICI/SD", "-ICI"], rotation = 0, rotation_mode="anchor", ha="center")
legend = plt.legend(loc='right', bbox_to_anchor=(1.7, .5), ncol=1, title="Cell Type", frameon=True)
plt.setp(legend.get_texts(),fontsize='8')
plt.setp(legend.get_title(),fontsize='12')
plt.savefig(os.path.join(fig_dir, "TNK_Comp_immune_barplot_by_ICI_status.png"), dpi=600, format="png", bbox_inches="tight")
plt.show()

In [ ]:
# stacked plot of cell type proportions per response
plot = pt.pl.coda.stacked_barplot(
    sccoda_data, 
    modality_key="coda", 
    feature_name="response",
    dpi=300,
)
plot.set_title("ICI Response")
plot.set_xticks([0,1], ["PD", "SD"], rotation = 0, rotation_mode="anchor", ha="center")
legend = plt.legend(loc='right', bbox_to_anchor=(1.7, .5), ncol=1, title="Cell Type", frameon=True)
plt.setp(legend.get_texts(),fontsize='8')
plt.setp(legend.get_title(),fontsize='12')
plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/TFM/Fig2", "TNK_Comp_immune_barplot_by_ICI_response.png"), dpi=600, format="png", bbox_inches="tight")
plt.show()

# Compositional Analysis - Non-responders Timepoints Comparison

In [ ]:
# set up dirs
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"
fig_dir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "timepoint_PD")
sc.settings.figdir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "timepoint_PD")

In [ ]:
# read anndata object
adata = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_TNK_annotated_13-04-24.h5ad"))

In [ ]:
# subset non-responders
adata = adata[adata.obs["response"] == "PD"].copy()
adata

## Model Setup & Inference

In [ ]:
# initiate scCODA model
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="Annotation_2.0",
    sample_identifier="sample",
    covariate_obs=["subproject", "patient", "timepoint", "response", "ICI_status", "Condition"],
        
)
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
pt.pl.coda.boxplots(sccoda_data, 
                    modality_key="coda", 
                    feature_name="timepoint", 
                    add_dots=True,
                    figsize=[6,6],
                    level_order=["SCR", "C02"],
                    cmap=["coral","darkviolet"]
                   )
plt.show()

In [ ]:
# prepare the model
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula="Condition", #condition = timepoint
    reference_cell_type="automatic",
)
sccoda_data["coda"]

In [ ]:
# run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda")
sccoda_data["coda"]

## Result Intepretation

In [ ]:
# see most relevant information
sccoda_model.summary(sccoda_data, modality_key="coda")

In [ ]:
sccoda_model.credible_effects(sccoda_data, modality_key="coda")

### Adjust FDR

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="coda", est_fdr=0.2)
sccoda_model.summary(sccoda_data, modality_key="coda")

## Save Results

In [ ]:
#sccoda_data.write_h5mu(os.path.join(work_dir, "data", "outputdata", "combined", "TNK_Combined_SCR_CO2_immune_scCODA_15-04-24.h5mu"))

In [ ]:
test_model = pt.tl.Sccoda()
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
effect = test_model.get_effect_df(sccoda_data, modality_key="coda")
effect

In [ ]:
sccoda_data["coda"].varm["intercept_df"]

## Visualization

In [ ]:
# visualize effects (significant changes in cell type composition)
plot=pt.pl.coda.effects_barplot(sccoda_data, 
                           modality_key="coda", 
                           #parameter="timepoint",
                           #figsize=[3,3],
                           dpi=300)
plt.xticks(fontsize=8) 
plt.yticks(fontsize=10)  
plt.savefig(os.path.join(work_dir, "figures", "TFM", "Fig2", "TNK_Comp_Significant_Log2FC_timepoint_PD.pdf"), dpi=300, format="pdf", bbox_inches="tight")

In [ ]:
# visualize effects (significant changes in cell type composition)
plot=pt.pl.coda.effects_barplot(sccoda_data, 
                           modality_key="coda", 
                           parameter="Final Parameter",
                           figsize=[3,3],
                           dpi=100)
plt.savefig(os.path.join(fig_dir, "TNK_Comp_Significant_Celltypes_Effects.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
# define cell types that have an effect
effect_cell_types = ["CD8 T TRM PreExhausted", "CD56hi CD16lo NK"]

In [ ]:
sccoda_data['coda'].var

In [ ]:
# some booxplot arguments to customize the plot
boxprops = dict(linestyle='-', linewidth=1, edgecolor='black')
flierprops = dict(marker='o', markerfacecolor='none', #markersize=12,
                  markeredgecolor='black')
medianprops = dict(linestyle='-', linewidth=.8, color='black')
#meanpointprops = dict(marker='D', markeredgecolor='black',markerfacecolor='black')
meanlineprops = dict(linestyle='--', linewidth=.8, color='black')


In [ ]:
# define cell types for plotting
cell_types = sccoda_data['coda'].var_names.tolist()
print(cell_types)

In [ ]:
# create df for plotting
feature_name=["Condition", "response", "patient"]
data=sccoda_data['coda']

sample_sums = np.sum(data.X, axis=1, keepdims=True)
X = data.X/sample_sums
value_name = "Proportion"

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
        merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)
if cell_types is not None:
    plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

In [ ]:
plot_df

In [ ]:
# overview of cell type distribution across patients
for cell_type in cell_types:

    if cell_type in effect_cell_types:
        significance = "*"
    else:
        significance = "ns"
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)

    sns.boxplot(
        x="patient", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="patient", 
        #liersize=1,
        palette="Spectral", 
        #order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="patient", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="Condition",  # Color by "patient" variable
        palette=["coral", "darkviolet"],  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=4,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type+" - "+significance)
    ax.set_xlabel('Patient')
    plt.yticks(fontsize = 10) 
    plt.xticks(fontsize = 10) 
    
    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Condition", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
      
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated" 
    
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_patient.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
plot = pt.pl.coda.boxplots(
        sccoda_data,
        modality_key="coda",
        feature_name="patient",
        figsize=(8,4),
        add_dots=False,
        plot_facets=False, 
        args_boxplot={"vert":True, 
                    "patch_artist":True, 
                    "meanline":True, 
                    "showmeans":True,
                    "boxprops":boxprops,
                    "flierprops":flierprops,
                    "medianprops":medianprops,
                    "meanprops":meanlineprops,
                    },
        cmap="Spectral"
        )
plt.xticks(fontsize=10)
plt.savefig(os.path.join(fig_dir, "Comp_all_boxplot_by_patient.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
effect["Final Parameter"]

In [ ]:
# overview of cell type distribution across timepoints

plt.rcParams.update({'font.size': 13})

for cell_type in cell_types:

    if cell_type in effect_cell_types:
    
        fig, ax = plt.subplots(figsize=(2,2), dpi=120)
        
        sns.boxplot(
            x="Condition", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type],
            hue="Condition", 
            #liersize=1,
            palette=["coral","darkviolet"], 
            order=["T0/-ICI","T1/+ICI"],
            ax=ax,
            vert=True, 
            patch_artist=True, 
            meanline=True, 
            showmeans=True,
            showfliers=False,
            boxprops=boxprops,
            #flierprops=flierprops,
            medianprops=medianprops,
            meanprops=meanlineprops
        )
            
        
        sns.swarmplot(
            x="Condition", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type], 
            hue="patient",  # Color by "patient" variable
            palette="Spectral",  # Choose a color palette
            dodge=False,  # Separate dots for each level of "patient
            #jitter=False
            ax=ax,
            size=5,
            edgecolor="black",
            linewidth=.5,
            alpha=1
        )
    
        
        plt.title(cell_type, fontweight="bold", pad=20)
        plt.xlabel('Patient', labelpad=10)
        plt.ylabel('Proportion', labelpad=10)
        plt.yticks(fontsize = 10) 
        plt.xticks(fontsize = 10) 
        #plt.set_xticks([0,1,2,3,4], ["PD_01", "PD_02", "PD_03", "SD_01", "PD_04"], ha="center", rotation=30)

        
        legend = plt.legend(loc='right', bbox_to_anchor=(2, .5), ncol=1, title="Patient", frameon=True)
        plt.setp(legend.get_title(),fontsize='12')
        plt.tight_layout()

        
        plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/ISCO24_poster/Fig5", "TNK_Comp_"+cell_type+"_boxplot_by_condition.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
# overview of cell type distribution across timepoints

plt.rcParams.update({'font.size': 13})

for cell_type in cell_types:

    if cell_type in effect_cell_types:
        significance = "*"
    else:
        significance = "ns"
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)
    
    sns.boxplot(
        x="Condition", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="Condition", 
        #liersize=1,
        palette=["coral","darkviolet"], 
        order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="Condition", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="patient",  # Color by "patient" variable
        palette="Spectral",  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=5,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type+" - "+significance)
    ax.set_xlabel('ICI Response')
    plt.yticks(fontsize = 10) 
    plt.xticks(fontsize = 10) 
    
    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Patient", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
    
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated"

        
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_condition.png"), dpi=600, format="png", bbox_inches="tight")


# Compositional Analysis - Response Comparison

In [ ]:
# set up dirs
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"
fig_dir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "response")
sc.settings.figdir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "response")

In [ ]:
# read anndata object
adata = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_TNK_annotated_13-04-24.h5ad"))

In [ ]:
# create condition variable
adata

## Model Setup & Inference

In [ ]:
# initiate scCODA model
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="Annotation_2.0",
    sample_identifier="sample",
    covariate_obs=["subproject", "patient", "timepoint", "response", "ICI_status", "Condition"],
        
)
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
# prepare the model
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula="response", #condition = timepoint
    reference_cell_type="automatic",
)
sccoda_data["coda"]

In [ ]:
# run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda")
sccoda_data["coda"]

## Result Intepretation

In [ ]:
# see most relevant information
sccoda_model.summary(sccoda_data, modality_key="coda")

In [ ]:
sccoda_model.credible_effects(sccoda_data, modality_key="coda")

### Adjust FDR

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="coda", est_fdr=0.2)
sccoda_model.summary(sccoda_data, modality_key="coda")

## Save Results

In [ ]:
#sccoda_data.write_h5mu(os.path.join(work_dir, "data", "outputdata", "combined", "TNK_Combined_SCR_CO2_immune_scCODA_15-04-24.h5mu"))

In [ ]:
test_model = pt.tl.Sccoda()
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
effect = test_model.get_effect_df(sccoda_data, modality_key="coda")
effect

In [ ]:
sccoda_data["coda"].varm["intercept_df"]

## Visualization

In [ ]:
# visualize effects (significant changes in cell type composition)
plot=pt.pl.coda.effects_barplot(sccoda_data, 
                           modality_key="coda", 
                           #parameter="timepoint",
                           #figsize=[3,3],
                           dpi=300)
plt.xticks(fontsize=8) 
plt.yticks(fontsize=10)  
plt.savefig(os.path.join(work_dir, "figures", "TFM", "Fig2", "TNK_Comp_Significant_Log2FC_timepoint_response.pdf"), dpi=300, format="pdf", bbox_inches="tight")

In [ ]:
'''
# visualize effects (significant changes in cell type composition)
plot=pt.pl.coda.effects_barplot(sccoda_data, 
                           modality_key="coda", 
                           parameter="Final Parameter",
                           figsize=[3,3],
                           dpi=100)
plt.savefig(os.path.join(fig_dir, "TNK_Comp_Significant_Celltypes_Effects.png"), dpi=600, format="png", bbox_inches="tight")
'''

In [ ]:
# define cell types that have an effect
effect_cell_types = []

In [ ]:
sccoda_data['coda'].var

In [ ]:
# some booxplot arguments to customize the plot
boxprops = dict(linestyle='-', linewidth=1, edgecolor='black')
flierprops = dict(marker='o', markerfacecolor='none', #markersize=12,
                  markeredgecolor='black')
medianprops = dict(linestyle='-', linewidth=.8, color='black')
#meanpointprops = dict(marker='D', markeredgecolor='black',markerfacecolor='black')
meanlineprops = dict(linestyle='--', linewidth=.8, color='black')


In [ ]:
# define cell types for plotting
cell_types = sccoda_data['coda'].var_names.tolist()
print(cell_types)

In [ ]:
# create df for plotting
feature_name=["Condition", "response", "patient"]
data=sccoda_data['coda']

sample_sums = np.sum(data.X, axis=1, keepdims=True)
X = data.X/sample_sums
value_name = "Proportion"

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
        merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)
if cell_types is not None:
    plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

In [ ]:
plot_df

In [ ]:
# overview of cell type distribution across patients
sns.set_style(rc = {'axes.facecolor': 'lightsteelblue'})

# define cell types that have an effect
effect_cell_types_ns = ["CD8 T Proliferative", "CD8 T TRM Effector", "CD8 T TRM PreExhausted", "CD8 T TRM Exhausted"]

for cell_type in cell_types:
    
    if cell_type in effect_cell_types_ns:

        fig, ax = plt.subplots(figsize=(2,2), dpi=120)
    
        sns.boxplot(
            x="Condition", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type],
            hue="Condition", 
            #order=["T0/-ICI","T1/+ICI"],
            ax=ax,
            vert=True, 
            patch_artist=True, 
            palette=["coral", "darkviolet"],
            meanline=True, 
            showmeans=True,
            showfliers=False,
            boxprops=boxprops,
            #flierprops=flierprops,
            medianprops=medianprops,
            meanprops=meanlineprops
        )
            
        
        sns.swarmplot(
            x="Condition", 
            y="Proportion", 
            data=plot_df[plot_df["Cell type"] == cell_type], 
            hue="response",  # Color by "patient" variable
            palette=["red", "blue"],# Choose a color palette
            dodge=False,  # Separate dots for each level of "patient
            #jitter=False
            ax=ax,
            size=4,
            edgecolor="black",
            linewidth=.5,
            alpha=1
        )

        sns.pointplot(
            data=plot_df[plot_df["Cell type"] == cell_type], 
            x="Condition", 
            y="Proportion", 
            hue="patient", 
            legend= False,
            palette="Spectral",
            linewidth=1.5,
            errwidth=1,
            markers='|',
            #errorbar=None,
            join=True
        )
        
        plt.title(cell_type, fontweight="bold", pad=20)
        plt.xlabel('Patient', labelpad=10)
        plt.ylabel('Proportion', labelpad=10)
        plt.yticks(fontsize = 10) 
        plt.xticks(fontsize = 10) 
        #plt.set_xticks([0,1,2,3,4], ["PD_01", "PD_02", "PD_03", "SD_01", "PD_04"], ha="center", rotation=30)

        
        legend = plt.legend(loc='right', bbox_to_anchor=(2, .5), ncol=1, title="Patient", frameon=True, facecolor="white")
        plt.setp(legend.get_title(),fontsize='12')
        plt.tight_layout()

        
        plt.savefig(os.path.join("/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/figures/ISCO24_poster/Fig5", "TNK_Comp_"+cell_type+"_boxplot_by_response.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
# overview of cell type distribution across patients
for cell_type in cell_types:

    if cell_type in effect_cell_types:
        significance = "*"
    else:
        significance = "ns"
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)

    sns.boxplot(
        x="response", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="response", 
        #order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        palette=["mistyrose", "lavender"], 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="response", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="Condition",  # Color by "patient" variable
        palette=["coral", "darkviolet"],  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=4,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type+" - "+significance)
    ax.set_xlabel('Response')
    plt.yticks(fontsize = 10) 
    plt.xticks(fontsize = 10) 
    
    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Condition", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
      
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated" 
    
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_response_col_by_condition.png"), dpi=600, format="png", bbox_inches="tight")

In [ ]:
# overview of cell type distribution across patients
for cell_type in cell_types:

    if cell_type in effect_cell_types:
        significance = "*"
    else:
        significance = "ns"
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)

    sns.boxplot(
        x="response", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="response", 
        #order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        palette=["mistyrose", "lavender"], 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="response", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="patient",  # Color by "patient" variable
        palette="Spectral",  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=4,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type+" - "+significance)
    ax.set_xlabel('Response')
    plt.yticks(fontsize = 10) 
    plt.xticks(fontsize = 10) 
    
    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Patient", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
      
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated" 
    
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_response_col_by_patient.png"), dpi=600, format="png", bbox_inches="tight")

# Compositional Analysis - ICI_status Comparison

In [ ]:
# set up dirs
work_dir = "/scratch_isilon/groups/singlecell/gdeuner/SERPENTINE/"
fig_dir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "ICI_status")
sc.settings.figdir = os.path.join(work_dir, "figures", "combined", "TNK", "compositional_analysis", "ICI_status")

In [ ]:
# read anndata object
adata = sc.read_h5ad(os.path.join(work_dir, "data", "outputdata", "combined", "Combined_SCR_CO2_TNK_annotated_13-04-24.h5ad"))

In [ ]:
# create condition variable
adata

## Model Setup & Inference

In [ ]:
# initiate scCODA model
sccoda_model = pt.tl.Sccoda()
sccoda_data = sccoda_model.load(
    adata,
    type="cell_level",
    generate_sample_level=True,
    cell_type_identifier="Annotation_2.0",
    sample_identifier="sample",
    covariate_obs=["subproject", "patient", "timepoint", "response", "ICI_status", "Condition"],
        
)
print(sccoda_data)
print(sccoda_data["coda"].X)
print(sccoda_data["coda"].obs)

In [ ]:
sccoda_data['coda'].obs.ICI_status.unique()

In [ ]:
order = ['-ICI', '+ICI/SD', '+ICI/PD']
sccoda_data['coda'].obs['ICI_status'] = pd.Categorical(sccoda_data['coda'].obs['ICI_status'], categories=order)

In [ ]:
# prepare the model
sccoda_data = sccoda_model.prepare(
    sccoda_data,
    modality_key="coda",
    formula="ICI_status", #condition = timepoint
    reference_cell_type="automatic",
)
sccoda_data["coda"]

In [ ]:
# run MCMC
sccoda_model.run_nuts(sccoda_data, modality_key="coda")
sccoda_data["coda"]

## Result Intepretation

In [ ]:
# see most relevant information
sccoda_model.summary(sccoda_data, modality_key="coda")

In [ ]:
sccoda_model.credible_effects(sccoda_data, modality_key="coda")

### Adjust FDR

In [ ]:
sccoda_model.set_fdr(sccoda_data, modality_key="coda", est_fdr=0.2)
sccoda_model.summary(sccoda_data, modality_key="coda")

## Save Results

In [ ]:
#sccoda_data.write_h5mu(os.path.join(work_dir, "data", "outputdata", "combined", "TNK_Combined_SCR_CO2_immune_scCODA_15-04-24.h5mu"))

In [ ]:
test_model = pt.tl.Sccoda()
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
test_model.get_intercept_df(sccoda_data, modality_key="coda")

In [ ]:
effect = test_model.get_effect_df(sccoda_data, modality_key="coda")
effect

In [ ]:
sccoda_data["coda"].varm["intercept_df"]

## Visualization

In [ ]:
sccoda_data['coda'].var_names

In [ ]:
# some booxplot arguments to customize the plot
boxprops = dict(linestyle='-', linewidth=1, edgecolor='black')
flierprops = dict(marker='o', markerfacecolor='none', #markersize=12,
                  markeredgecolor='black')
medianprops = dict(linestyle='-', linewidth=.8, color='black')
#meanpointprops = dict(marker='D', markeredgecolor='black',markerfacecolor='black')
meanlineprops = dict(linestyle='--', linewidth=.8, color='black')


In [ ]:
# define cell types for plotting
cell_types = sccoda_data['coda'].var_names.tolist()
print(cell_types)

In [ ]:
# create df for plotting
feature_name=["Condition", "response", "patient", "ICI_status"]
data=sccoda_data['coda']

sample_sums = np.sum(data.X, axis=1, keepdims=True)
X = data.X/sample_sums
value_name = "Proportion"

count_df = pd.DataFrame(X, columns=data.var.index, index=data.obs.index).\
        merge(data.obs[feature_name], left_index=True, right_index=True)
plot_df = pd.melt(count_df, id_vars=feature_name, var_name="Cell type", value_name=value_name)
if cell_types is not None:
    plot_df = plot_df[plot_df["Cell type"].isin(cell_types)]

In [ ]:
plot_df

In [ ]:
# define cell types that have an effect
effect1 = ["CD4 T CM/EarlyActivated", "CD4 T Follicular Helper", "CD4 T Regulatory", "CD8 T Proliferative", "CD8 T TRM Effector", "CD8 T TRM Exhausted", "CD8 T TRM PreExhausted", "CD56dim CD16hi NK"]
effect2 = ["CD4 T CM/EarlyActivated", "CD4 T Central Memory", "CD4 T Follicular Helper", "CD4 T Naive", "CD8 T TRM PreExhausted", "CD56dim CD16lo NK"]

effect_dict_1 = { # -ICI vs. +ICI/SD
    'CD4 T CM/EarlyActivated' : '*', 
    'CD4 T Central Memory' : 'ns',
    'CD4 T Follicular Helper' : '*', 
    'CD4 T Helper-like' : 'ns', 
    'CD4 T ISG+' : 'ns',
    'CD4 T Naive' : 'ns', 
    'CD4 T Proliferative' : 'ns', 
    'CD4 T Regulatory' : '*',
    'CD8 T Effector' : 'ns', 
    'CD8 T MAIT' : 'ns', 
    'CD8 T Naive' : 'ns', 
    'CD8 T Proliferative' : '*',
    'CD8 T TRM Effector' : '*', 
    'CD8 T TRM Exhausted' : '*', 
    'CD8 T TRM PreExhausted' : '*',
    'CD56dim CD16hi NK' : '*', 
    'CD56hi CD16lo NK' : 'ns', 
    'Cycling γδ T-like' : 'ns',
    'NKT-like' : 'ns', 
    'γδ T-like' : 'ns'
}

effect_dict_2 = { # -ICI vs. +ICI/PD
    'CD4 T CM/EarlyActivated' : '*', 
    'CD4 T Central Memory' : '*',
    'CD4 T Follicular Helper' : '*', 
    'CD4 T Helper-like' : 'ns', 
    'CD4 T ISG+' : 'ns',
    'CD4 T Naive' : '*', 
    'CD4 T Proliferative' : 'ns', 
    'CD4 T Regulatory' : 'ns',
    'CD8 T Effector' : 'ns', 
    'CD8 T MAIT' : 'ns', 
    'CD8 T Naive' : 'ns', 
    'CD8 T Proliferative' : 'ns',
    'CD8 T TRM Effector' : 'ns', 
    'CD8 T TRM Exhausted' : 'ns', 
    'CD8 T TRM PreExhausted' : '*',
    'CD56dim CD16hi NK' : 'ns', 
    'CD56hi CD16lo NK' : '*', 
    'Cycling γδ T-like' : 'ns',
    'NKT-like' : 'ns', 
    'γδ T-like' : 'ns'
}

In [ ]:
for cell_type in cell_types:
    
    fig, ax = plt.subplots(figsize=(2,2), dpi=120)

    sns.boxplot(
        x="ICI_status", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type],
        hue="ICI_status", 
        #order=["T0/-ICI","T1/+ICI"],
        ax=ax,
        vert=True, 
        patch_artist=True, 
        palette=["white", "blue", "red"], 
        meanline=True, 
        showmeans=True,
        showfliers=False,
        boxprops=boxprops,
        #flierprops=flierprops,
        medianprops=medianprops,
        meanprops=meanlineprops
    )
        
    
    sns.swarmplot(
        x="ICI_status", 
        y="Proportion", 
        data=plot_df[plot_df["Cell type"] == cell_type], 
        hue="patient",  # Color by "patient" variable
        palette="Spectral",  # Choose a color palette
        dodge=False,  # Separate dots for each level of "patient
        #jitter=False
        ax=ax,
        size=4,
        edgecolor="black",
        linewidth=.5,
        alpha=1
    )

    #ax.set_xticks([])
    ax.set_title(cell_type)
    ax.set_xlabel('ICI Status')
    ymax = plot_df[plot_df["Cell type"] == cell_type]["Proportion"].max()
    ax.plot([0, 0, 2, 2], [ymax + .08, ymax + .09, ymax + .09, ymax + .08], lw=1, color='black')
    ax.text(1, ymax+.085, f" {effect_dict_2[cell_type]}", ha='center', va='bottom', size=8, color='black')
    ax.plot([0, 0, 1, 1], [ymax + .05, ymax + .06, ymax + .06, ymax + .05], lw=1, color='black')
    ax.text(0.5, ymax+.055, f" {effect_dict_1[cell_type]}", ha='center', va='bottom', size=8, color='black')
    #ax.set_ylim([0, ymax+.1])
    plt.yticks(fontsize = 9) 
    plt.xticks(fontsize = 9) 

    legend = plt.legend(loc='right', bbox_to_anchor=(1.8, .5), ncol=1, title="Patient", frameon=True)
    plt.setp(legend.get_title(),fontsize='12')
    plt.tight_layout()
      
    if cell_type == "CD4 T CM/EarlyActivated":
        cell_type = "CD4 T CM-EarlyActivated" 
    
    plt.savefig(os.path.join(fig_dir, "TNK_Comp_"+cell_type+"_boxplot_by_ICI_status.png"), dpi=600, format="png", bbox_inches="tight")